In [60]:
import pandas as pd
from ydata_profiling import ProfileReport
import numpy as np
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [61]:
todrop = ['PassengerId','Ticket','Name','Cabin']
train = train.drop(columns= todrop,axis=1)
PId = test['PassengerId']
test = test.drop(columns= todrop,axis=1)

In [62]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [63]:
# profile = ProfileReport(train, title="Profiling Report",html = {'style':{'full_width':True}})
# profile.to_file("output.html")

In [64]:
train = train.dropna(subset=['Embarked'])
train['Age'] = train['Age'].fillna(train['Age'].median())
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())

In [65]:
train = train.drop_duplicates(keep='first')
train.shape

(773, 8)

In [66]:
train['Survived'].value_counts()

Survived
0    455
1    318
Name: count, dtype: int64

In [67]:
def encode(df):
    df['Sex'].replace({'male': 0, 'female': 1}, inplace=True)
    df['Embarked'].replace({'S': 0, 'C': 1, 'Q':2}, inplace=True)

In [68]:
encode(train)
encode(test)

In [69]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
X = train.drop("Survived", axis=1)
y = train["Survived"]
oversampler = RandomOverSampler()
X_resampled, y_resampled = oversampler.fit_resample(X, y)

In [70]:
from sklearn.preprocessing import StandardScaler
index_column = X_resampled.index
scaler = StandardScaler()
data_standardized = scaler.fit_transform(X_resampled)
data_standardized_df = pd.DataFrame(data_standardized, columns=X_resampled.columns, index=index_column)
print(data_standardized_df)

       Pclass       Sex       Age     SibSp     Parch      Fare  Embarked
0    0.918090 -0.867970 -0.510749  0.506965 -0.524363 -0.510528 -0.581783
1   -1.428713  1.152114  0.654101  0.506965 -0.524363  0.589941  1.047209
2    0.918090  1.152114 -0.219537 -0.553583 -0.524363 -0.498928 -0.581783
3   -1.428713  1.152114  0.435692  0.506965 -0.524363  0.277444 -0.581783
4    0.918090 -0.867970  0.435692 -0.553583 -0.524363 -0.496780 -0.581783
..        ...       ...       ...       ...       ...       ...       ...
905 -0.255312  1.152114  0.945313  0.506965 -0.524363 -0.188293 -0.581783
906 -0.255312  1.152114  0.872510 -0.553583  0.689811 -0.300001 -0.581783
907 -0.255312  1.152114  0.362888  0.506965  0.689811 -0.076585 -0.581783
908  0.918090 -0.867970 -2.039615  0.506965  1.903986 -0.281527 -0.581783
909 -1.428713 -0.867970 -1.311584  0.506965  1.903986  1.427180 -0.581783

[910 rows x 7 columns]


In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_standardized_df, y_resampled, test_size=0.33, random_state=42)


In [72]:
from xgboost import XGBClassifier
model = XGBClassifier()  
model.fit(X_train, y_train)
model.score(X_test,y_test)

0.8338870431893688

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
m1 = LogisticRegression()
m1.fit(X_train,y_train)
m1.score(X_test,y_test)

0.8006644518272426

In [74]:
test_sd = scaler.fit_transform(test)
pred = m1.predict(test_sd)
pred

C:\Users\dream\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [76]:
sur = pd.DataFrame({'Survived': pred})
PId = pd.DataFrame({'PassengerId': PId})

In [77]:
result = PId.merge(sur, left_index=True, right_index=True)
result.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [78]:
result.to_csv("submission.csv",index=False)